### Pip Install Operations


In [1]:
! pip install spektral
! pip install ogb
! pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 267.3 kB/s eta 0:00:00a 0:00:01
  Using cached lxml-4.9.2-cp311-cp311-macosx_11_0_arm64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 5.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00ta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 6.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

## Setup


In [4]:
import torch
from torch_geometric.data import Data
import random

from spektral.datasets.ogb import OGB
from spektral.transforms import AdjToSpTensor, GCNFilter
from ogb.nodeproppred import Evaluator, NodePropPredDataset

from torch_geometric.nn import GCNConv
from torch.nn import BCEWithLogitsLoss
import torch.nn.functional as F

from torch_geometric.utils import negative_sampling

from itertools import product, combinations

from sklearn.metrics import recall_score
import itertools

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_edge_percentage = 0.2
V_percentage = 0.05
SEED = 42

torch.manual_seed(SEED)

### Training Related Setup

## Dataset Related


### Loading the dataset

In [6]:
# import ogbn ogbn-arxiv


dataset_name = "ogbn-arxiv"
ogb_dataset = NodePropPredDataset(dataset_name)
dataset = OGB(ogb_dataset, transforms=[GCNFilter(), AdjToSpTensor()])

### Converting dataset from TF to Torch

In [7]:
# convert tf dataset to torch tensor



# Get the node features, edge indices, and labels
features = dataset[0].x
edge_indices = dataset[0].a.indices
labels = dataset[0].y

# Convert TensorFlow tensors to PyTorch Tensors
features_torch = torch.from_numpy(features)
edge_indices_torch = torch.from_numpy(edge_indices.numpy().T).long()  # Transpose to fit PyG's edge_index format and convert to long
labels_torch = torch.from_numpy(labels)

# Create a PyTorch Geometric Data object
data = Data(x=features_torch, edge_index=edge_indices_torch, y=labels_torch)

### Applying dataset splits



In [8]:


# # # # # #
# Getting V and V_new
# # # # # #

# Assume that `data` is your PyTorch Geometric graph object.
# data = ...

# Get the number of nodes in your graph.
num_nodes = data.num_nodes

# Create a random permutation of indices [0, 1, 2, ..., num_nodes-1].
perm = torch.randperm(num_nodes)

# Calculate the index at which to split the permutation.
split_idx = int(num_nodes * V_percentage)

# Split the permutation into indices for V (95%) and V_new (5%).
V = perm[:split_idx]
V_new = perm[split_idx:]

# V and V_new are now the indices of the nodes in the 95% and 5% splits, respectively.

# ------> For node classification





In [9]:


# # # # # #
# Splitting edges to training and validation edges
# # # # # #



# Assuming your data is in this format
# data = Data(x=features_torch, edge_index=edge_indices_torch, y=labels_torch)

# Get the number of edges
num_edges = data.edge_index.size(1)

# Create a list of indices representing the edges
edge_indices = list(range(num_edges))

# Shuffle the indices randomly
random.shuffle(edge_indices)

# Define the percentage of edges to be used for training
num_train_edges = int(train_edge_percentage * num_edges)

# Split the indices into two sets: for training and validation
train_edge_indices = edge_indices[:num_train_edges]
val_edge_indices = edge_indices[num_train_edges:]

# Function to create a new edge_index tensor based on selected indices
def create_edge_index_subset(edge_index, selected_indices):
    return edge_index[:, selected_indices]

# Create new edge_index tensors for training and validation
E_train = create_edge_index_subset(data.edge_index, train_edge_indices)
E_val = create_edge_index_subset(data.edge_index, val_edge_indices)

# Now, 'edge_index_train' contains the edges for the training set,
# and 'edge_index_val' contains the edges for the validation set.


## Model Related


### Classical Backbone Model (GCN)

In [10]:


# Define a simple GNN model
class GCN(torch.nn.Module):
    def __init__(self, num_features):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 256)
        self.conv2 = GCNConv(256, 256)
        self.conv3 = GCNConv(256, 256)

        self.scoring = torch.nn.Sequential(
            torch.nn.Linear(2 * 256, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1)
        )

    def forward(self, data, edge_index):
        x = self.conv1(data.x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv3(x, edge_index)
        return x

    def decode(self, z, indices):
        start, end = indices
        edge_features = torch.cat([z[start], z[end]], dim=1)
        return self.scoring(edge_features).squeeze(-1)


def bpr_loss(pos_logit, neg_logit):
    return -F.logsigmoid(pos_logit - neg_logit).sum()


## Training, Validation Test


### Train

In [14]:


def train(model,V, data, train_edges, val_edges, optimizer, patience=10, epochs = 1000, test_active = True):

  # Define some initial best validation loss as infinity
  best_val_loss = float('inf')
  epochs_no_improve = 0

  # Training loop
  data, train_edges, val_edges = data.to(device), train_edges.to(device), val_edges.to(device)
  for epoch in range(epochs):  # 1000 epochs
      print("epoch ", epoch)

      model.train()
      optimizer.zero_grad()

      z_train = model(data, train_edges)  # embeddings for training edges
      pos_edge_index = train_edges  # positive examples
      neg_edge_index = negative_sampling(edge_index=pos_edge_index, num_nodes=z_train.size(0))  # negative examples

      #print("pos_edge_index.shape: ", pos_edge_index.shape)
      pos_logit = model.decode(z_train, pos_edge_index)
      neg_logit = model.decode(z_train, neg_edge_index)

      loss = bpr_loss(pos_logit, neg_logit)

      loss.backward()
      optimizer.step()

      print("train loss: ", loss.item())

      if test_active:
        res = test(model, V, val_edges, 50)
        print("recall@50: ", res)

      # Validation:
      if (epoch +1) % 5 == 0:
        # validation function calls model.eval(), calculating both val loss & recall@50
        val_loss, recall_50 = validation(model, data, val_edges, 50)
        print(f'Validation Loss: {val_loss}, Recall@50: {recall_50}')

        # Check if early stopping conditions are met
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f'Early stopping triggered after {epoch+1} epochs.')
                break




### Validation


In [13]:


def validation(model, nodes, val_edges, z, k=50):
    #model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        

        # Convert V to a boolean tensor for faster lookup.
        v_mask = torch.zeros(num_nodes, dtype=torch.bool)
        v_mask[nodes] = True

        # Assume val_edges contains the validation edges (it should be a 2 x num_val_edges tensor)
        # val_edges = ...

        # Check if both nodes of each edge in val_edges are in V
        source_nodes = val_edges[0, :]
        target_nodes = val_edges[1, :]
        can_exist_in_V = v_mask[source_nodes] & v_mask[target_nodes]

        # Filter the edges that can exist in V
        valid_edges_in_V = val_edges[:, can_exist_in_V]
        positive_pairs = valid_edges_in_V

        # --- Generating negative pairs ---

        # Find the unique starting nodes in val_edges
        start_nodes = torch.unique(val_edges[0, :])

        # Generate all possible pairs from start_nodes to all nodes in V
        all_possible_pairs = torch.tensor(list(product(start_nodes.tolist(), V.tolist())))

        # Remove the existing edges in val_edges from all_possible_pairs to create the negative pairs
        existing_pairs = valid_edges_in_V.t()
        negative_pairs = []
        for pair in all_possible_pairs:
            if not any(torch.all(pair == existing_pair, dim=0) for existing_pair in existing_pairs):
                negative_pairs.append(pair)

        negative_pairs = torch.stack(negative_pairs).t()


          # Negative examples for validation

        pos_logit_val = model.decode(z, positive_pairs)
        neg_logit_val = model.decode(z, negative_pairs)

        val_loss = bpr_loss(pos_logit_val, neg_logit_val)

    return val_loss.item()

### Test

In [1]:
def test(model, nodes, val_edges, z, k=50):
    model.eval()  # Set the model to evaluation mode

    # Take 5 samples from val_edges as positive examples
    val_edges = val_edges[:, torch.randint(val_edges.size(1), (5,))]

    with torch.no_grad():

        # Convert V to a boolean tensor for faster lookup.
        v_mask = torch.zeros(num_nodes, dtype=torch.bool)
        v_mask[nodes] = True
        v_mask = v_mask.to(device)

        # Assume val_edges contains the validation edges (it should be a 2 x num_val_edges tensor)
        # val_edges = ...

        # Check if both nodes of each edge in val_edges are in V
        source_nodes = val_edges[0, :]
        target_nodes = val_edges[1, :]
        can_exist_in_V = v_mask[source_nodes] & v_mask[target_nodes]

        # Filter the edges that can exist in V
        valid_edges_in_V = val_edges[:, can_exist_in_V]
        positive_pairs = valid_edges_in_V

        # --- Generating negative pairs ---

        # Find the unique starting nodes in val_edges
        start_nodes = torch.unique(val_edges[0, :])

        # Generate all possible pairs from start_nodes to all nodes in V
        all_possible_pairs = torch.tensor(list(product(start_nodes.tolist(), V.tolist())))

        # Remove the existing edges in val_edges from all_possible_pairs to create the negative pairs
        existing_pairs = valid_edges_in_V.t()
        negative_pairs = []
        negative_pairs.to(device)
        
        for pair in all_possible_pairs:
            if not any(torch.all(pair == existing_pair, dim=0) for existing_pair in existing_pairs):
                negative_pairs.append(pair)

        negative_pairs = torch.stack(negative_pairs).t()


          # Negative examples for validation

        positive_scores = model.decode(z, positive_pairs)
        negative_scores = model.decode(z, negative_pairs)

        # Combine positive and negative scores
        all_scores = torch.cat([positive_scores, negative_scores])

        # Indicate which edges are positive (1 for positive, 0 for negative)
        positive_edge_indicator = torch.tensor([1]*val_edges.size(1) + [0]*negative_pairs.size(1))


        recall_per_node = calculate_recall_per_node(start_nodes, all_scores, positive_edge_indicator, k)

        # Calculate the average recall over all starting nodes
        recall = recall_per_node.mean()

        return recall.item()



## Utils

### Recall calculation


In [ ]:


# recall_at_k_per_node(model, z, val_edges, k, unique_nodes, data.edge_index)
def recall_at_k():

  # Count all relevant target nodes for the nodes
  total_rel = None

  # Count number of relevant edges at k
  total_rel_k = None

  # Ratio them and average

  recall = total_rel_k / total_rel
  recall_avg = None







  # get val nodes
  val_nodes = torch.unique(val_edges)

  # get all the real edges from the val nodes: pos_v
  mask = torch.isin(edge_index, val_nodes).any(dim=0)
  positive_edges = edge_index[:, mask]

  # get all possible edges: all_v
  all_edges_val = list(itertools.product(val_nodes.tolist(), unique_nodes.tolist()))
  all_edges_val = torch.tensor(all_edges_val, dtype=torch.long).t().contiguous()

  # get scores for all possible edges
  scores = model.decode(z, all_edges_val)

  # Get top k scores and their corresponding edges
  _, top_k_indices = scores.topk(k, largest=True)
  top_k_edges = all_edges_val[:, top_k_indices.cpu()]

  # check how many of them in top50
  top_k_edges = set( tuple( sorted((int(n1), int(n2))) ) for n1, n2 in zip(top_k_edges[0], top_k_edges[1]))
  positive_edges = set(tuple(sorted((int(n1), int(n2)))) for n1, n2 in zip(positive_edges[0], positive_edges[1]))

  print("top_k_edges: \n", top_k_edges)
  print("positive_edges \n", positive_edges)

  # calculate recall@k
  num_hits = len(top_k_edges & graph2_edges)
  recall_at_K = num_hits / len(positive_edges)

  return recall_at_K




In [2]:
def calculate_recall_per_node(start_nodes, all_scores, positive_edge_indicator, K):
    """
    Calculate recall for each individual starting node using tensor operations.
    
    Parameters:
    - start_nodes: Tensor of shape [num_edges], containing the starting node of each edge.
    - all_scores: Tensor of shape [num_edges], containing scores for each edge.
    - positive_edge_indicator: Tensor of shape [num_edges], containing 1 for positive edges and 0 for negative edges.
    - K: The number of top edges to consider for calculating recall.
    
    Returns:
    - recall_per_node: Dictionary with nodes as keys and recall as values.
    """

    # Sort scores in descending order
    sorted_indices = torch.argsort(all_scores, descending=True)

    # Apply sorting to start nodes and positive indicators
    sorted_start_nodes = start_nodes[sorted_indices]
    sorted_positive_indicators = positive_edge_indicator[sorted_indices]

    # Count the number of positive edges in top K for each unique start node
    unique_start_nodes, counts = torch.unique(sorted_start_nodes, return_counts=True)
    top_k_mask = (counts > K).int() * K + (counts <= K).int() * counts
    accumulated_positive = sorted_positive_indicators.cumsum(dim=0)
    total_positive_in_top_k = accumulated_positive[top_k_mask - 1]

    # Calculate recall for each unique start node
    recalls = total_positive_in_top_k.float() / K

    # Create a dictionary mapping start node to recall
    recall_per_node = {node.item(): recall.item() for node, recall in zip(unique_start_nodes, recalls)}

    return recall_per_node

### TuneUP: Synthesizing tail nodes

In [ ]:
from torch_geometric.utils import degree

def renormalize(edge_index, num_nodes):
    # Convert to PyTorch tensor for calculation
    edge_index = edge_index.clone().detach()
    
    # Calculate degree and create Degree Matrix D
    row, col = edge_index
    deg = degree(row, num_nodes, dtype=edge_index.dtype)
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt.masked_fill_(deg_inv_sqrt == float('inf'), 0)

    # Renormalize
    row, col = edge_index
    edge_weight = deg_inv_sqrt[row] * deg_inv_sqrt[col]

    return edge_index, edge_weight


def random_edge_sampler(data, percent):
    edge_index = data.edge_index
    num_nodes = data.num_nodes

    num_edges = edge_index.size(1)
    perm = torch.randperm(num_edges)
    preserve_nnz = int(num_edges * percent)

    # Indices for kept edges
    kept_indices = perm[:preserve_nnz]
    kept_edges = edge_index[:, kept_indices]
    kept_edges, kept_weights = renormalize(kept_edges, num_nodes)
    data_kept = Data(edge_index=kept_edges, edge_attr=kept_weights)

    # Indices for dropped edges
    dropped_indices = perm[preserve_nnz:]
    dropped_edges = edge_index[:, dropped_indices]
    dropped_edges, dropped_weights = renormalize(dropped_edges, num_nodes)
    data_dropped = Data(edge_index=dropped_edges, edge_attr=dropped_weights)

    return data_kept, data_dropped


"""
## USAGE:

# percent: rate of edges to keep
data_kept, data_dropped = random_edge_sampler(data, percent)

## INPUT:
Data(x=[169343, 128], edge_index=[2, 1335586], y=[169343, 1])

## RETURNS:
(Data(edge_index=[2, 468243]), Data(edge_index=[2, 962188]))

"""

# Execution 



In [24]:
model = GCN(128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.001)  # L2 regularization

train(model, V, data, train_edges=E_train, val_edges=E_val, optimizer=optimizer)


epoch  0


In [10]:
A = {"a": 1, "b": 2}

a = A.values()
a = list(a)

# turn a to np array
a = np.array(a)

[1, 2]

In [20]:
v_mask = torch.zeros(num_nodes, dtype=torch.bool)
v_mask.size()

torch.Size([169343])

In [21]:
num_nodes

169343

In [22]:
data.

Data(x=[169343, 128], edge_index=[2, 1335586], y=[169343, 1])

In [13]:
import torch

# Define tensors
E_all = torch.tensor([
    [1, 2, 3, 4, 11, 12],
    [5, 6, 7, 8, 13, 14]
], dtype=torch.int)

B = torch.tensor([
    [2, 3, 9, 11, 15],
    [6, 7, 10, 13, 16]
], dtype=torch.int)

# Check if a GPU is available and if so, use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
E_all = E_all.to(device)
B = B.to(device)

# Compute the pairwise equality
pairwise_equality = torch.eq(E_all.unsqueeze(2), B.unsqueeze(1))

# Determine the columns where all rows are True (i.e., both elements in column are equal)
column_equality = torch.all(pairwise_equality, dim=0)

# Extract the intersecting columns
intersection = B[:, column_equality.any(dim=0)]

# Remove intersection from B
B_without_intersection = B[:, ~column_equality.any(dim=0)]

# Display the intersection and B without intersection
print("Intersection:")
print(intersection.cpu())
print("B without intersection:")
print(B_without_intersection.cpu())


Intersection:
tensor([[ 2,  3, 11],
        [ 6,  7, 13]], dtype=torch.int32)
B without intersection:
tensor([[ 9, 15],
        [10, 16]], dtype=torch.int32)


In [14]:
import torch

# Assume start_nodes and V are one-dimensional tensors representing sets of nodes
start_nodes = torch.tensor([1, 2, 3, 4], dtype=torch.int)
V = torch.tensor([5, 6, 7, 8], dtype=torch.int)

# Check if a GPU is available and if so, use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
start_nodes = start_nodes.to(device)
V = V.to(device)

# Create all possible pairs tensor using broadcasting
all_possible_pairs = torch.stack(torch.meshgrid(start_nodes, V), dim=-1).reshape(-1, 2).t()

# Display all possible pairs
print(all_possible_pairs.cpu())


tensor([[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
        [5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8]], dtype=torch.int32)


/opt/homebrew/Caskroom/miniconda/base/envs/TUI/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


data